# COMPSCI 589 HW1

#### Name: Haochen Wang

### SECTION 0: Load Libraries

In [332]:
import sklearn.model_selection
import scipy
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
from operator import itemgetter
from collections import Counter

### SECTION 2: Evaluating Decision Tree

In [333]:
# Import Document

house_file = open('house_votes_84.csv', encoding='utf-8-sig')
csvreader = csv.reader(house_file)
houserows = []
for row in csvreader:
        houserows.append(row)

hvcat = houserows[0]
# print(housevotecat)

housevotedat = np.array([[int(element) for element in row] for row in houserows[1:]])

#print(len(housevotedat))

In [334]:
# Node Class
class Treenode:
    label = ""
    type = ""
    subattributelist = []
    

    def __init__(self, label, type):
        self.label = label
        self.type = type


In [335]:
# Define helper functions that I use in decision tree.

def same(column):
    return all(item == column[0] for item in column)

def majority(column):
    return np.argmax(np.bincount(column))

# test = np.array([1,1,3,3,11,11])
# same(test)
# majority(test)

In [336]:
# Define three test criteria: 

def entropy(col):
    dictcounter = dict(Counter(col))
    values = list(dictcounter.values())
    ent = 0
    for value in values:
        k = (value/sum(values))
        print(k)
        ent += -k*math.log(k,2)
    return ent
# ID3 - Entropy Gain 

def id3gain(father, list, listattribution):
    best = listattribution[0]
    
    return best

# C4.5 - Entrophy Ratio 
# CART - Gini Impurity
test =  np.array([1,1,0,0,0,0,0,0,0,0,0,0])
cc = dict(Counter(test))
dd = list(cc.values())
entropy(test)

0.09090909090909091
0.9090909090909091


0.4394969869215134

In [337]:
# Decision Tree
def decisiontree(dataset, listattributes, type='id3'):
    if type == "id3":
        criteria = 1
    elif type == "c4.5":
        criteria = 2
    elif type == "cart":
        criteria = 3

    node = Treenode("-1","node")

    if same([row[-1] for row in dataset]):
        node.type = "leaf"
        node.label = row[-1]
        return node
    
    if len(listattributes) == 0:
        node.type = "leaf"
        node.label = majority(dataset.T[-1])
        return node

    return